# **Lab 3: Categorical Data**


 ## Exercise 1:

The dataset we will be using contains data related to second hand car sales. Your goal is to build a regression model with regularisation to predict accurately the sale price.

You will be loading the dataset from this link:
https://raw.githubusercontent.com/aso-uts/labs_datasets/main/36106-mlaa/lab03/ex1/car_details.csv

The steps are:

1.   Load and Explore Dataset
2.   Data Preparation
3.   Split Datasets
4.   Assess Baseline Model
5.   Train Elastic Model

---
### 0. Setup Environment

In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [3]:
# Do not modify this code

import os
import sys
from pathlib import Path

COURSE = "36106"
LAB = "lab03"
EXE = "ex01"
DATA = "data"

lab_path = f"{COURSE}/labs/{LAB}/{EXE}"
root_path = "./"

if os.getenv("COLAB_RELEASE_TAG"):

    from google.colab import drive
    from pathlib import Path

    print("###### Install required Python packages ######")
    ! pip install -r https://raw.githubusercontent.com/aso-uts/labs_datasets/main/36106-mlaa/requirements.txt



print("\n###### Setting up folders ######")
folder_path = f"{root_path}/{lab_path}/"
root_path = Path(folder_path)
root_path.mkdir(parents=True, exist_ok=True)
print(f"\nYou can now save your data files in:")
%cd {folder_path}


###### Install required Python packages ######

###### Setting up folders ######

You can now save your data files in:
/content/36106/labs/lab03/ex01


### 1. Load and Explore Dataset

**[1.1]** Import the pandas, numpy and altair packages

In [4]:
import pandas as pd
import numpy as np
import altair as alt

**[1.2]** Create a variable called `file_url` containing the link to the CSV file

In [6]:
file_url = 'https://raw.githubusercontent.com/aso-uts/labs_datasets/main/36106-mlaa/lab03/ex1/car_details.csv'

**[1.3]** Load the dataset into dataframe called `df`

In [7]:
df = pd.read_csv(file_url)

**[1.4]** Display the first 5 rows of `df`

In [8]:
df.head(5)

,name,year,selling_price,km_driven,fuel,seller_type,transmission,owner
0,Maruti 800 AC,2007,60000,70000,Petrol,Individual,Manual,First Owner
1,Maruti Wagon R LXI Minor,2007,135000,50000,Petrol,Individual,Manual,First Owner
2,Hyundai Verna 1.6 SX,2012,600000,100000,Diesel,Individual,Manual,First Owner
3,Datsun RediGO T Option,2017,250000,46000,Petrol,Individual,Manual,First Owner
4,Honda Amaze VX i-DTEC,2014,450000,141000,Diesel,Individual,Manual,Second Owner


**[1.5]** Display the last 5 rows of `df`

In [10]:
df.tail(5)

,name,year,selling_price,km_driven,fuel,seller_type,transmission,owner
4335,Hyundai i20 Magna 1.4 CRDi (Diesel),2014,409999,80000,Diesel,Individual,Manual,Second Owner
4336,Hyundai i20 Magna 1.4 CRDi,2014,409999,80000,Diesel,Individual,Manual,Second Owner
4337,Maruti 800 AC BSIII,2009,110000,83000,Petrol,Individual,Manual,Second Owner
4338,Hyundai Creta 1.6 CRDi SX Option,2016,865000,90000,Diesel,Individual,Manual,First Owner
4339,Renault KWID RXT,2016,225000,40000,Petrol,Individual,Manual,First Owner


**[1.6]** Display the dimensions of `df`

In [11]:
df.shape

(4340, 8)

**[1.7]** Display the descriptive statistics of `df`


In [12]:
df.describe()

,year,selling_price,km_driven
count,4340.000000,4.340000e+03,4340.000000
mean,2013.090783,5.041273e+05,66215.777419
std,4.215344,5.785487e+05,46644.102194
min,1992.000000,2.000000e+04,1.000000
25%,2011.000000,2.087498e+05,35000.000000
50%,2014.000000,3.500000e+05,60000.000000
75%,2016.000000,6.000000e+05,90000.000000
max,2020.000000,8.900000e+06,806599.000000


**[1.8]** Display the summary of `df`

In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4340 entries, 0 to 4339
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   name           4340 non-null   object
 1   year           4340 non-null   int64 
 2   selling_price  4340 non-null   int64 
 3   km_driven      4340 non-null   int64 
 4   fuel           4340 non-null   object
 5   seller_type    4340 non-null   object
 6   transmission   4340 non-null   object
 7   owner          4340 non-null   object
dtypes: int64(3), object(5)
memory usage: 271.4+ KB


### 2. Data Preparation

**[2.1]** Create a copy of our dataframe called `df_cleaned`

In [25]:
df_cleaned = df.copy()

**[2.2]** Save all the columns names of `df` that are numerical type into a list called `num_cols`

In [30]:
num_cols = df.select_dtypes(include=np.number).columns.tolist()

**[2.3]** Save all the columns names of `df` that are ordinal type into a list called `ord_cols`

In [20]:
ord_cols = df.select_dtypes(include=['category']).columns.tolist()

**[2.4]** Save all the columns names of `df` that are categorical type into a list called `cat_cols`

In [21]:
cat_cols = df.select_dtypes(include=['object']).columns.tolist()

**[2.5]** Iterate through all columns from `ord_cols` and print their list of unique values

In [22]:
for col in ord_cols:
  print(col)
  print(df_cleaned[col].unique())

**[2.6]** Create a dictionary mapping the values of `owner` column to an ordinal value

In [23]:
owner_mapper = {
    "Test Drive Car": 0,
    "First Owner": 1,
    "Second Owner": 2,
    "Third Owner": 3,
    "Fourth & Above Owner": 4
}
owner_mapper

{'Test Drive Car': 0,
 'First Owner': 1,
 'Second Owner': 2,
 'Third Owner': 3,
 'Fourth & Above Owner': 4}

**[2.7]** Replace the values of the `owner` column to their corresponsing ordinal value

In [26]:
df_cleaned["owner"] = df_cleaned["owner"].replace(owner_mapper)
df_cleaned["owner"]

,owner
0,1
1,1
2,1
3,1
4,2
...,...
4335,2
4336,2
4337,2
4338,1


**[2.8]** Iterate through all columns from `cat_cols` and print their list of unique values

In [31]:
for col in cat_cols:
  print(col)
  print(df_cleaned[col].unique())


name
['Maruti 800 AC' 'Maruti Wagon R LXI Minor' 'Hyundai Verna 1.6 SX' ...
 'Mahindra Verito 1.5 D6 BSIII'
 'Toyota Innova 2.5 VX (Diesel) 8 Seater BS IV'
 'Hyundai i20 Magna 1.4 CRDi']
fuel
['Petrol' 'Diesel' 'CNG' 'LPG' 'Electric']
seller_type
['Individual' 'Dealer' 'Trustmark Dealer']
transmission
['Manual' 'Automatic']
owner
[1 2 4 3 0]


**[2.9]** Perform one-hot encoding on all categorical columns and save them into a separate DataFrame called `df_cat`

In [28]:
df_cat = pd.get_dummies(df_cleaned[cat_cols])
df_cat

,owner,name_Ambassador CLASSIC 1500 DSL AC,name_Ambassador Classic 2000 Dsz,name_Ambassador Grand 1800 ISZ MPFI PW CL,name_Audi A4 1.8 TFSI,name_Audi A4 2.0 TDI,name_Audi A4 2.0 TDI 177 Bhp Premium Plus,name_Audi A4 3.0 TDI Quattro,name_Audi A4 30 TFSI Technology,name_Audi A4 35 TDI Premium,...,fuel_CNG,fuel_Diesel,fuel_Electric,fuel_LPG,fuel_Petrol,seller_type_Dealer,seller_type_Individual,seller_type_Trustmark Dealer,transmission_Automatic,transmission_Manual
0,1,False,False,False,False,False,False,False,False,False,...,False,False,False,False,True,False,True,False,False,True
1,1,False,False,False,False,False,False,False,False,False,...,False,False,False,False,True,False,True,False,False,True
2,1,False,False,False,False,False,False,False,False,False,...,False,True,False,False,False,False,True,False,False,True
3,1,False,False,False,False,False,False,False,False,False,...,False,False,False,False,True,False,True,False,False,True
4,2,False,False,False,False,False,False,False,False,False,...,False,True,False,False,False,False,True,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4335,2,False,False,False,False,False,False,False,False,False,...,False,True,False,False,False,False,True,False,False,True
4336,2,False,False,False,False,False,False,False,False,False,...,False,True,False,False,False,False,True,False,False,True
4337,2,False,False,False,False,False,False,False,False,False,...,False,False,False,False,True,False,True,False,False,True
4338,1,False,False,False,False,False,False,False,False,False,...,False,True,False,False,False,False,True,False,False,True


**[2.10]** Save all the numerical features (using `num_cols` and `ord_cols`) into a variable called `X`

In [32]:


X = df_cleaned[num_cols + ord_cols]


**[2.11]** Combine the one-hot encoded columns contained in `df_cat` into `X`

In [ ]:
X = pd.concat([X, df_cat], axis=1)
X

,selling_price,year,km_driven,owner,fuel_CNG,fuel_Diesel,fuel_Electric,fuel_LPG,fuel_Petrol,seller_type_Dealer,seller_type_Individual,seller_type_Trustmark Dealer,transmission_Automatic,transmission_Manual
0,60000,2007,70000,1,False,False,False,False,True,False,True,False,False,True
1,135000,2007,50000,1,False,False,False,False,True,False,True,False,False,True
2,600000,2012,100000,1,False,True,False,False,False,False,True,False,False,True
3,250000,2017,46000,1,False,False,False,False,True,False,True,False,False,True
4,450000,2014,141000,2,False,True,False,False,False,False,True,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4335,409999,2014,80000,2,False,True,False,False,False,False,True,False,False,True
4336,409999,2014,80000,2,False,True,False,False,False,False,True,False,False,True
4337,110000,2009,83000,2,False,False,False,False,True,False,True,False,False,True
4338,865000,2016,90000,1,False,True,False,False,False,False,True,False,False,True


# 3. Split Datasets

**[3.1]** Extract the target variable into a variable called `y`

Note: If you are stuck in previous steps, you can dowload the content of df_cleaned here: https://raw.githubusercontent.com/aso-uts/labs_datasets/main/36106-mlaa/lab03/ex1/df_cleaned.csv

In [33]:
url  = 'https://raw.githubusercontent.com/aso-uts/labs_datasets/main/36106-mlaa/lab03/ex1/df_cleaned.csv'
y = df_cleaned['selling_price']

**[3.2]** Create the test set (called `X_test` and `y_test`) by filtering rows only from the year 2018

In [34]:

X_test = X[df_cleaned['year'] == 2018]
y_test = y[df_cleaned['year'] == 2018]


**[3.3]** Create the validation set (called `X_val` and `y_val`) by filtering rows only from the year 2017

In [35]:


X_val = X[df_cleaned['year'] == 2017]
y_val = y[df_cleaned['year'] == 2017]


**[3.4]** Create the training set (called `X_train` and `y_train`) by keeping rows only prior to the year 2017

In [36]:

X_train = X[df_cleaned['year'] < 2017]
y_train = y[df_cleaned['year'] < 2017]


### 4. Assess Baseline Model

**[4.1]** Import the DummyRegressor module from sklearn

In [37]:
from sklearn.dummy import DummyRegressor

**[4.2]** Instantiate the Dummy class into a variable called `base_reg` and fit it on the training set it

In [38]:
base_reg = DummyRegressor()
base_reg.fit(X_train, y_train)

DummyRegressor()

**[4.3]** Import the RMSE and MAE from sklearn

In [39]:
from sklearn.metrics import mean_squared_error, mean_absolute_error

**[4.4]** Display the RMSE and MAE scores of this baseline model

In [41]:
import numpy as np
y_pred = base_reg.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
mae = mean_absolute_error(y_test, y_pred)

print(f"RMSE: {rmse}")
print(f"MAE: {mae}")

RMSE: 978199.2527188903
MAE: 561842.2370948711


### 5. Train Elastic Model

**[5.1]** Import the ElasticNet module from sklearn

In [42]:
from sklearn.linear_model import ElasticNet

**[5.2]** Instantiate the ElasticNet class into a variable called `elasticnet_reg` and fit it on the training set

In [43]:
elasticnet_reg = ElasticNet()
elasticnet_reg.fit(X_train, y_train)

ElasticNet()

**[5.3]** Use the trained model to predict the outcome on `X_train` and save the results into a variable called `y_preds`

In [44]:
y_preds = elasticnet_reg.predict(X_train)

**[5.4]** Display the RMSE and MAE scores on the training set

In [45]:


import numpy as np
rmse_train = np.sqrt(mean_squared_error(y_train, y_preds))
mae_train = mean_absolute_error(y_train, y_preds)

print(f"RMSE (Training Set): {rmse_train}")
print(f"MAE (Training Set): {mae_train}")


RMSE (Training Set): 5.207948909151648e-07
MAE (Training Set): 3.027108771217328e-07


**[5.5]** Save the predictions on the validation set into a variable called `y_val_preds` and display the RMSE and MAE scores

In [46]:

import numpy as np
y_val_preds = elasticnet_reg.predict(X_val)

rmse_val = np.sqrt(mean_squared_error(y_val, y_val_preds))
mae_val = mean_absolute_error(y_val, y_val_preds)

print(f"RMSE (Validation Set): {rmse_val}")
print(f"MAE (Validation Set): {mae_val}")


RMSE (Validation Set): 1.029066330963717e-06
MAE (Validation Set): 5.475069419181078e-07


**[5.6]** Display the predictions versus the target line charts on the training set

In [47]:


import pandas as pd
import altair as alt

# Create a DataFrame for plotting
predictions_df = pd.DataFrame({'Target': y_train, 'Predictions': y_preds})

# Create the Altair chart
chart = alt.Chart(predictions_df).mark_line().encode(
    x=alt.X('Target', title='Target'),
    y=alt.Y('Predictions', title='Predictions')
).properties(
    title='Predictions vs. Target on Training Set'
)

# Display the chart
chart.display()


alt.Chart(...)

**[5.7]** Display the predictions versus the target line charts on the validation set

In [48]:


import pandas as pd
# Create a DataFrame for plotting
predictions_df_val = pd.DataFrame({'Target': y_val, 'Predictions': y_val_preds})

# Create the Altair chart
chart_val = alt.Chart(predictions_df_val).mark_line().encode(
    x=alt.X('Target', title='Target'),
    y=alt.Y('Predictions', title='Predictions')
).properties(
    title='Predictions vs. Target on Validation Set'
)

# Display the chart
chart_val.display()


alt.Chart(...)